# TD4 : Inference

## Libraries importation

In [48]:
import torch
import numpy as np
import parametrization_cookbook.torch as pc

## SBM function

In [71]:
def SBM(n_nodes = 10, n_classes = 3, prob_classes = None, alpha = None, self_loop = False):
    """Generate SBM"""
    if prob_classes is None:
        prob_classes = np.ones(shape=n_classes) / n_classes
    if alpha is None:
        alpha = np.ones(shape=(n_classes, n_classes)) / 3
        np.fill_diagonal(alpha, 2/3)
    # Create Z, class vector
    Z = np.random.choice([i for i in range(n_classes)], size=n_nodes, p=prob_classes)
    # Generate bernouilli law
    alpha_expanded = alpha[Z][:, Z]
    X = np.random.binomial(n=1, p=alpha_expanded)
    if not self_loop:
        np.fill_diagonal(X, 0)
    return Z, X

## Variables

In [72]:
n = 100
K = 3
param_gamma = pc.VectorSimplex(dim = K-1, shape=(n, ))
param_pi = pc.VectorSimplex(dim=K-1)
param_alpha = pc.RealBounded01(shape=(K,K))
prob_classes = np.array([1/3, 1/3, 1/3])
alpha = np.array([[2/3, 1/3, 1/3],
         [1/3, 2/3, 1/3],
         [1/3, 1/3, 2/3]])

Z, X = SBM(n_nodes = n, n_classes=K, prob_classes=prob_classes, alpha=alpha)
X_tensor = torch.tensor(X, dtype=torch.float32)
Z_tensor = torch.tensor(Z, dtype=torch.int64)

In [73]:
type(Z), type(X), type(param_gamma), type(param_pi), type(param_alpha)

(numpy.ndarray,
 numpy.ndarray,
 parametrization_cookbook.torch.VectorSimplex,
 parametrization_cookbook.torch.VectorSimplex,
 parametrization_cookbook.torch.RealBounded01)

In [74]:
def ELBO_from_unconstrained_params(gamma_prime, pi_prime, alpha_prime):
    """Return the ELBO from the unconstrained parameters"""
    gamma = param_gamma.reals1d_to_params(gamma_prime)
    pi = param_pi.reals1d_to_params(pi_prime)
    alpha = param_alpha.reals1d_to_params(alpha_prime)

    entropy = - torch.einsum("ik,ik->", gamma, torch.log(gamma))
    expectation_log_likelihood = (torch.einsum("ik,k->", gamma, torch.log(pi)) 
                                  + torch.einsum("ik, jl,ij,kl->", gamma, gamma, X_tensor, torch.log(alpha))
                                  + torch.einsum("ik, jl,ij,kl->", gamma, gamma, 1 - X_tensor, torch.log(1-alpha))) 
    return entropy + expectation_log_likelihood

In [75]:
gp = torch.randn(param_gamma.size).clone().detach().requires_grad_(True)
pp = torch.randn(param_pi.size).clone().detach().requires_grad_(True)
ap = torch.randn(param_alpha.size).clone().detach().requires_grad_(True)

In [76]:
optim = torch.optim.Adam([gp, pp, ap], lr=0.01)

for i in range(5000):
    optim.zero_grad()
    elbo = ELBO_from_unconstrained_params(gp, pp, ap)
    (-elbo).backward()
    optim.step()
    if i % 1000 == 0:
        print(f"i = {i}, - elbo = {-elbo.item()}")

i = 0, - elbo = 7356.4443359375
i = 1000, - elbo = 6605.58740234375
i = 2000, - elbo = 6529.68896484375
i = 3000, - elbo = 6495.92822265625
i = 4000, - elbo = 6494.9912109375


In [77]:
param_gamma.reals1d_to_params(gp)[:10, :]

tensor([[1.1625e-04, 3.6981e-05, 9.9985e-01],
        [1.3205e-04, 9.9975e-01, 1.1733e-04],
        [9.9978e-01, 2.2140e-04, 1.2698e-06],
        [9.9977e-01, 2.3250e-04, 8.0793e-07],
        [5.8603e-05, 9.9989e-01, 5.5231e-05],
        [9.8436e-05, 9.9988e-01, 2.0864e-05],
        [9.9925e-01, 7.5067e-04, 2.3048e-06],
        [1.5965e-04, 5.0752e-05, 9.9979e-01],
        [7.6727e-05, 5.3790e-05, 9.9987e-01],
        [1.5002e-04, 2.5013e-05, 9.9982e-01]], grad_fn=<SliceBackward0>)

In [78]:
param_alpha.reals1d_to_params(ap)

tensor([[0.6136, 0.3292, 0.3163],
        [0.3499, 0.6224, 0.3324],
        [0.3223, 0.3117, 0.6600]], grad_fn=<MulBackward0>)

In [79]:
param_pi.reals1d_to_params(pp)

tensor([0.2597, 0.3404, 0.4000], grad_fn=<SoftmaxBackward0>)